Imports and set some global variables

In [2]:
import shutil
import os
import subprocess
import sys
from datetime import datetime, timedelta
import glob
import ftplib
import tarfile

base_dir = '/home/rmcd/test/'
work_dir = base_dir + 'NHM-PRMS_CONUS'
ftp_server = 'ftpint.usgs.gov'
ftp_path = 'pub/cr/co/denver/BRR-CR/pub/markstro/onhm/'
ftp_login = 'anonymous'
ftp_pw = ''
python_module_path = "/work/markstro/operat/repos/onhm-runners/"

Create the NHM-PRMS workspace directory and subdirectories. This downloads param file, control file, etc. It creates the PRMS workspace for the ONHM.

In [3]:
# Remove the old work space
if os.path.exists(work_dir):
    shutil.rmtree(work_dir)
    
os.chdir(base_dir)

filename = 'NHM-PRMS_CONUS.tar.gz'

# Get the tar ball from ftp
ftp = ftplib.FTP(ftp_server) 
ftp.login(ftp_login, ftp_pw) 
ftp.cwd(ftp_path)
ftp.retrbinary("RETR " + filename, open(filename, 'wb').write)
ftp.quit()

# unzip and untar
tf = tarfile.open(base_dir + filename)
tf.extractall()

Install thredds server into NHM_PRMS workspace


Not implemented yet.

Make the historical PRMS run offline. There are still lots of issues (ie, updats to the HRUs and the current CBH files end in 2014), but goto

/work/markstro/operat/setup/init_file/NHM-PRMS_CONUS

to run "run_for_init.sh" that will create the prms_save_XXXX_XX_XX.init file.

Right now, download an init file from the ftp server to illustrate that the initial init file comes from outside of what is going on here.

In [4]:
os.chdir(work_dir)

filename = 'prms_save_2019-03-21.init'

# Get the init file from ftp
ftp = ftplib.FTP(ftp_server) 
ftp.login(ftp_login, ftp_pw) 
ftp.cwd(ftp_path)
ftp.retrbinary("RETR " + filename, open(filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Here starts the operational loop. The following steps are repeated everyday.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Figure out the dates.

In [5]:
yesterday = datetime.today() - timedelta(1)
sixtydays = yesterday - timedelta(60)
initdate = sixtydays - timedelta(1)

model_start_date = sixtydays
model_end_date = yesterday

fetch_start_date = initdate
fetch_end_date = yesterday

# The init_start_date is one day after whatever the current init_date is. Get this from the
# file in the workspace.
os.chdir(work_dir)
for file in glob.glob("*.init"):
    ;
    
tok = file.split('_')
foo = tok[2]
tok = foo.split('.')
date_str = tok[0]

init_start_date = datetime.strptime(date_str, '%Y-%m-%d')
init_end_date = sixtydays

print('model_start_date = ' + model_start_date.strftime('%Y-%m-%d'))
print('model_end_date = ' + model_end_date.strftime('%Y-%m-%d'))
print('fetch_start_date = ' + fetch_start_date.strftime('%Y-%m-%d'))
print('fetch_end_date = ' + fetch_end_date.strftime('%Y-%m-%d'))
print('init_start_date = ' + init_start_date.strftime('%Y-%m-%d'))
print('init_end_date = ' + init_end_date.strftime('%Y-%m-%d'))


model_start_date = 2019-03-30
model_end_date = 2019-05-29
fetch_start_date = 2019-03-29
fetch_end_date = 2019-05-29
init_start_date = 2019-03-21
init_end_date = 2019-03-30


Run the Fetcher-Parser to bring the CBH netcdf up to yesterday. This copy from stage business simulates that Fetcher-Parser.

Not implemented yet, but pull from ftp as a place holder.

In [9]:
os.chdir(work_dir)

# docker pull for ofp
arg = 'docker pull nhmusgs/ofp'
arg_l = arg.split() # need this for whatever reason; don't move arg.split into the func call below

subprocess.check_output(arg_l)


b'Using default tag: latest\nlatest: Pulling from nhmusgs/ofp\nDigest: sha256:c00ba1406555f0c1b538ab67e3c819c63a7830da4aa747f75c093f7f1b068c26\nStatus: Image is up to date for nhmusgs/ofp:latest\n'

In [11]:
# run the nhmusgs/ofp docker.  Here the arguement -d2 specifies the number of days to retrieve. If Gridment is updated in this case it would retrieve yesteday and the day before.
arg = 'docker run -v' + work_dir + ':/code/ofp/Output nhmusgs/ofp:latest -d2'
arg_l = arg.split() # need this for whatever reason; don't move arg.split into the func call below

subprocess.check_output(arg_l)

KeyboardInterrupt: 

In [ ]:
# os.chdir(work_dir)

# filename = 'new.nc'

# # Get the init file from ftp
# ftp = ftplib.FTP(ftp_server) 
# ftp.login(ftp_login, ftp_pw) 
# ftp.cwd(ftp_path)
# ftp.retrbinary("RETR " + filename, open(filename, 'wb').write)
# ftp.quit()

# shutil.copy(stage_dir + 'stage_2/new.nc', work_dir)

Run from onhm-runners ncf2cbh.ncf2cbh.py. This will write PRMS format CBH files that contain
data from 2019-01-01 up through yesterday.

It's here, but needs more work!

In [ ]:
# Not sure about the best way to handle importation of modules in jupyter,
# but this seems to work
# My PRMS python utilities are in onhm-runners/prms_utils
if python_module_path not in sys.path:
    sys.path.append(python_module_path)

# Spell out full paths -- don't use string variables -- don't know why.
%run '/work/markstro/operat/repos/onhm-runners/ncf2cbh/ncf2cbh.py' '/work/markstro/operat/setup/test/NHM-PRMS_CONUS/'

Run PRMS-NHM in the workspace. This run should always go from the day after the current
init file up through yesterday.

The next step works to run PRMS, but the start_time and end_time logic is not figured out yet.

In [ ]:
os.chdir(work_dir)
#arg = 'docker run --rm -v ' + work_dir + ':/work prms:5.0.0 ./NHM-PRMS.control'

fa = './NHM-PRMS.control -set init_vars_from_file 1 -set save_vars_to_file 0'

# figure out the following from start_date and end_date
sa = ' -set start_date ' + str(model_start_date.year) + ',' + str(model_start_date.month) + ',' + str(model_start_date.day) + ',0,0,0'
print(sa)
ea = ' -set end_date ' + str(model_end_date.year) + ',' + str(model_end_date.month) + ',' + str(model_end_date.day) + ',0,0,0'
print(ea)
ia = ' -set var_init_file prms_save_' + initdate.strftime('%Y-%m-%d') + '.init'
print(ia)

arg = 'docker run --rm -v ' + work_dir + ':/work nhmusgs/prms:5.0.0 ' + fa + sa + ea + ia
arg_l = arg.split() # need this for whatever reason; don't move arg.split into the func call below

subprocess.check_output(arg_l)

Determine if the run completed successfully and evaluate for goodness.


This step is not yet implemented.

If the current model run was good, run prms_outputs2_ncf.py to convert the output to ncf. This will eventually run in a container.

In [ ]:
# Not sure about the best way to handle importation of modules in jupyter,
# but this seems to work
# My PRMS python utilities are in onhm-runners/prms_utils

if python_module_path not in sys.path:
    sys.path.append(python_module_path)

# Spell out full paths -- don't use string variables -- don't know why.
%run '/work/markstro/operat/repos/onhm-runners/out2ncf/prms_outputs2_ncf.py' '/work/markstro/operat/setup/test/NHM-PRMS_CONUS/'

If the current model run was good, run PRMS-NHM in the workspace to update the init file from whatever date it is at to yesterday.

In [ ]:
os.chdir(work_dir)

# get the list of existing init files.
old_init_file_name_list = []
os.chdir(work_dir)
for file in glob.glob("*.init"):
    old_init_file_name_list.append(file)
    
print(old_init_file_name_list)

# Create the new init file

fa = './NHM-PRMS.control -set init_vars_from_file 1 -set save_vars_to_file 1'

# figure out the following from start_date and end_date
sa = ' -set start_date ' + str(init_start_date.year) + ',' + str(init_start_date.month) + ',' + str(init_start_date.day) + ',0,0,0'
print(sa)
ea = ' -set end_date ' + str(init_end_date.year) + ',' + str(init_end_date.month) + ',' + str(init_end_date.day) + ',0,0,0'
print(ea)

ia = ' -set var_init_file prms_save_' + init_start_date.strftime('%Y-%m-%d') + '.init' + ' -set var_save_file prms_save_' + init_end_date.strftime('%Y-%m-%d') + '.init'
print(ia)

arg = 'docker run --rm -v ' + work_dir + ':/work nhmusgs/prms:5.0.0 ' + fa + sa + ea + ia
arg_l = arg.split() # need this for whatever reason; don't move arg.split into the func call below

subprocess.check_output(arg_l)

Check to make sure that the PRMS init model run worked. If it did, erase the old init files.



Done for this day's step. Wait until tomorrow and go back to the date computation step and repeat subsequent steps. 